## Part 1: Database and Jupyter Notebook Setup

```bash
mongoimport --db uk_food --collection establishments --file Resources/establishments.json --jsonArray --drop
```

In [1]:
import json
from configparser import ConfigParser
import pprint

import pymongo
import pandas as pd
import pymongo
import matplotlib.pyplot as plt


In [2]:
def load_configuration(f='database.ini', section='mongodb'):
    parser = ConfigParser()
    parser.read(f)

    config = { }

    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            config[param[0]] = param[1]
    else:
        raise Exception(f'section {section} not found in {filename}')

    return config


In [3]:
client = pymongo.MongoClient('mongodb://localhost:27017/')

db = client['uk_food']
coll = db['establishments']

for db_name in client.list_database_names():
    print(db_name)
    d = client[db_name]
    for c in d.list_collection_names():
        print(f'\t - {c}')



admin
	 - system.version
config
	 - system.sessions
fruit_db
	 - fruits
local
	 - startup_log
met
	 - artifacts
myDatabase
	 - books
	 - students
	 - orders
petsitly_marketing
	 - customer_list
test
	 - music
	 - artifacts
test_db
	 - first_collection
uk_food
	 - establishments


In [4]:
with open('Resources/establishments.json', encoding='utf8') as f:
    data = f.read()

In [5]:
df = pd.DataFrame(json.loads(data))

In [6]:
pprint.pprint(coll.find_one({}))

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67e63be0194e3f972c724db0'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

## Part 2: Update the Database

In [ ]:
greenwich = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

establishments = coll

for c in establishments.find({'BusinessType': 'Restaurant/Cafe/Canteen'}).limit(10):
    print(c['BusinessType'], c['BusinessTypeID'])


print()

greenwich['BusinessTypeID'] = establishments.find_one({'BusinessType': 'Restaurant/Cafe/Canteen'})['BusinessTypeID']

greenwich

Restaurant/Cafe/Canteen 1
Restaurant/Cafe/Canteen 1
Restaurant/Cafe/Canteen 1
Restaurant/Cafe/Canteen 1
Restaurant/Cafe/Canteen 1
Restaurant/Cafe/Canteen 1
Restaurant/Cafe/Canteen 1
Restaurant/Cafe/Canteen 1
Restaurant/Cafe/Canteen 1
Restaurant/Cafe/Canteen 1



{'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

In [8]:
# The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

print(establishments.count_documents({'LocalAuthorityName': 'Dover'}))

establishments.delete_many({'LocalAuthorityName': 'Dover'})

print(establishments.count_documents({'LocalAuthorityName': 'Dover'}))



994
0


In [10]:
# Some of the number values are stored as strings, when they should be stored as numbers.
# Use the update_many() method to convert the string values to floats. The following fields need to be converted:

# - geocode.longitude
# - geocode.latitude

establishments.update_many(
    {'geocode.longitude': {'$type': 'string'}},
    [{'$set': {'geocode.longitude': {'$toDouble': '$geocode.longitude'}}}]
)

establishments.update_many(
    {'geocode.latitude': {'$type': 'string'}},
    [{'$set': {'geocode.latitude': {'$toDouble': '$geocode.latitude'}}}]
)

# Check that the values have been converted to floats by using the find_one() method on the collection. The geocode field should now contain float values.
print(establishments.find_one({})['geocode'])


{'longitude': 1.195625, 'latitude': 51.083812}


In [ ]:
# Use update_many to convert RatingValue to integer numbers (already is....)

print(list(establishments.find({}).limit(5)))


[{'_id': ObjectId('67e63be0194e3f972c725097'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': 5, 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': 1.195625, 'latitude': 51.083812}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'retu

# Part 3: Exploratory Analysis

In [14]:
# Use count_documents to display the number of documents contained in the result.
print(establishments.count_documents({}))

38785


In [15]:
# Display the first document in the results using pprint

pprint.pprint(establishments.find_one({}))


{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67e63be0194e3f972c725097'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,


In [17]:
# Convert the result to a Pandas DataFrame.
# Print the number of rows in the DataFrame.
# Print the first 10 rows of the DataFrame.

df = pd.DataFrame(list(establishments.find({})))
num_rows = df.shape[0]
print(f'Number of rows: {num_rows}')

print(df.shape)
print(df.head(10))


Number of rows: 38785
(38785, 28)
                        _id   FHRSID  ChangesByServerID  \
0  67e63be0194e3f972c725097  1043695                  0   
1  67e63be0194e3f972c725098   647177                  0   
2  67e63be0194e3f972c725099   289353                  0   
3  67e63be0194e3f972c72509c   289352                  0   
4  67e63be0194e3f972c72509d   289560                  0   
5  67e63be0194e3f972c72509e   987206                  0   
6  67e63be0194e3f972c72509f   344689                  0   
7  67e63be0194e3f972c7250a0   894592                  0   
8  67e63be0194e3f972c7250a1  1043701                  0   
9  67e63be0194e3f972c7250a2   805702                  0   

  LocalAuthorityBusinessID                         BusinessName  \
0             PI/000073616                         The Pavilion   
1             PI/000041489                  Wear Bay Bowls Club   
2             PI/000002468  St Marys COE (aided) Primary School   
3             PI/000002460                      

In [49]:
# Which establishments have a hygiene score equal to 20?

hygiene_score_20 = establishments.find({'scores.Hygiene': 20})
print(list(hygiene_score_20))

# What is the average rating value of each establishment type?
# Use the aggregate method to calculate the average rating value of each establishment type.

pipeline = [
    {
        '$group': {
            '_id': '$BusinessType',
            'average_rating': {'$avg': '$RatingValue'}
        }
    }
]

results = establishments.aggregate(pipeline)

# Convert the results to a Pandas DataFrame and print the first 10 rows.

df = pd.DataFrame(list(results))

print(df.head(10))



[{'_id': ObjectId('67e63be1194e3f972c726bc9'), 'FHRSID': 110681, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': '4029', 'BusinessName': 'The Chase Rest Home', 'BusinessType': 'Caring Premises', 'BusinessTypeID': 5, 'AddressLine1': '5-6 Southfields Road', 'AddressLine2': 'Eastbourne', 'AddressLine3': 'East Sussex', 'AddressLine4': '', 'PostCode': 'BN21 1BU', 'Phone': '', 'RatingValue': '0', 'RatingKey': 'fhrs_0_en-gb', 'RatingDate': '2021-09-23T00:00:00', 'LocalAuthorityCode': '102', 'LocalAuthorityName': 'Eastbourne', 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings', 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk', 'scores': {'Hygiene': 20, 'Structural': 20, 'ConfidenceInManagement': 20}, 'SchemeType': 'FHRS', 'geocode': {'longitude': 0.27694, 'latitude': 50.769705}, 'RightToReply': '', 'Distance': 4613.888288172291, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, '

In [54]:
# Which establishments in London have a rating value of 4 or higher?

pipeline = [
    {
        '$match': {
            'LocalAuthorityName': 'London',
            'RatingValue': {'$gte': 4}
        }
    },
    {
        '$group': {
            '_id': '$BusinessType',
            'average_rating': {'$avg': '$RatingValue'}
        }
    }
]

results = establishments.aggregate(pipeline)

print(list(results))

[]
